In [1]:
import psycopg2 # need install
import json
from annotation import Annotator

In [2]:
# loading from config file
with open("config.json", "r") as file:
    config = json.load(file)


In [3]:
db_config = config["db"]
conn = psycopg2.connect(
            host=db_config["host"],
            dbname=db_config["dbname"],
            user=db_config["user"],
            password=db_config["pwd"]
    )
if conn:
    print("Connected to database %s sucessfully!" % db_config["dbname"])
else:
    print("Connection failed.")

Connected to database db4031 sucessfully!


```python
#query with template
query_temp = "SELECT * FROM nation WHERE nation.n_regionkey = %s;"
vals = (1,)
query = cursor.mogrify(query_temp, vals)
```

```python
# Explain params?
cursor.execute("EXPLAIN (analyze, buffers, verbose, format json) " + query)
cursor.execute("EXPLAIN (FORMAT JSON) " + query)
```

In [4]:
if conn:
    cursor = conn.cursor()
    query = "SELECT COUNT(c.c_custkey) FROM customer as c, nation as n  WHERE c.c_nationkey = n.n_nationkey"
    cursor.execute("EXPLAIN (FORMAT JSON)" + query)
    plans = cursor.fetchall()

In [5]:
plans

[([{'Plan': {'Node Type': 'Aggregate',
     'Strategy': 'Plain',
     'Partial Mode': 'Finalize',
     'Parallel Aware': False,
     'Async Capable': False,
     'Startup Cost': 5545.45,
     'Total Cost': 5545.46,
     'Plan Rows': 1,
     'Plan Width': 8,
     'Plans': [{'Node Type': 'Gather',
       'Parent Relationship': 'Outer',
       'Parallel Aware': False,
       'Async Capable': False,
       'Startup Cost': 5545.23,
       'Total Cost': 5545.44,
       'Plan Rows': 2,
       'Plan Width': 8,
       'Workers Planned': 2,
       'Single Copy': False,
       'Plans': [{'Node Type': 'Aggregate',
         'Strategy': 'Plain',
         'Partial Mode': 'Partial',
         'Parent Relationship': 'Outer',
         'Parallel Aware': False,
         'Async Capable': False,
         'Startup Cost': 4545.23,
         'Total Cost': 4545.24,
         'Plan Rows': 1,
         'Plan Width': 8,
         'Plans': [{'Node Type': 'Hash Join',
           'Parent Relationship': 'Outer',
          

In [6]:
x = Annotator()

In [7]:
print(x.wrapper(plans))

Step 1: Perform sequential scan on table customer as c. 
Step 2: Perform sequential scan on table nation as n. 
Step 3: Perform hashing on table nation. 
Step 4: Perform a hash join on tables customer and nation under the condition (c.c_nationkey = n.n_nationkey) to get intermediate table T1. 
Step 5: Perform aggregate on table T1 to get intermediate table T2. 
Step 6: Perform gather on table T2 to get intermediate table T3. 
Step 7: Perform aggregate on table T3 to get the final result.
